Test: affichage trop lent necessité de conv en video

In [ ]:
import matplotlib.pyplot as plt
import gym
from gym.wrappers.monitoring.video_recorder import VideoRecorder
import os
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage

%matplotlib inline
path_project = os.path.abspath(os.path.join("__file__", ".."))
env = gym.make("ALE/SpaceInvaders-v5",render_mode="rgb_array")
env.reset()
before_training = os.path.join(path_project, "before_training.mp4")

for i in range(1000):
    
    img=plt.imshow(env.render())
    img.set_data(env.render())
    env.step(env.action_space.sample()) # take a random action

env.close()

In [ ]:
import matplotlib.pyplot as plt
import gym
from gym.wrappers.monitoring.video_recorder import VideoRecorder
import os
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage
import copy
import numpy as np
%matplotlib inline
fps=25
T=1000*fps
path_project = os.path.abspath(os.path.join("__file__", ".."))
video = os.path.join(path_project, "before_training.mp4")
env = gym.make("ALE/SpaceInvaders-v5",render_mode="rgb_array")


env.reset()

lst=[]
done = False
i=0
while i<T and done==False: 
    lst.append(copy.deepcopy(env.render()))
    state, reward, done, _,_ =env.step(env.action_space.sample()) # take a random action
    i+=1
def make_frame(t):
    """Returns an image of the frame for time t."""
    # ... create the frame with any library here ...
    return lst[int(t*fps)] # (Height x Width x 3) Numpy array
animation = VideoClip(make_frame, duration=len(lst)//fps) # T-second clip
# export as a video file
animation.write_videofile(video, fps=fps)
env.close()


In [ ]:
import matplotlib.pyplot as plt
import gym
from gym.wrappers.monitoring.video_recorder import VideoRecorder
from os import path
from moviepy.editor import VideoClip
from moviepy.video.io.bindings import mplfig_to_npimage
import copy
import os
import numpy as np
import random
import pickle #rick
%matplotlib inline
fps=50
path_project = os.path.abspath(os.path.join("__file__", ".."))
q_object=path_project+"/q_table"
video = os.path.join(path_project, "after_training.mp4")
env = gym.make("ALE/SpaceInvaders-v5",render_mode="rgb_array")

#Setting the hyperparameters
alpha = 0.7 #learning rate                 
discount_factor = 0.8             
true_epsilon= 0.001              
max_epsilon = 0.5
min_epsilon = 0.001         
decay = 1.1         

train_episodes = 250
max_steps = 40*fps #maybe remove q_table file if you reduce this value and pass epsilon to 1
print(max_steps)

def make_frame(t):
    """Returns an image of the frame for time t."""
    # ... create the frame with any library here ...
    return lst[int(t*fps)] # (Height x Width x 3) Numpy array

#STEP 1 - load Q-table or create it

if path.isfile(q_object):
    fileQ=open(q_object, 'rb')
    Q = pickle.load(fileQ)
    fileQ.close()
    while Q.size/env.action_space.n<max_steps:
        Q=np.vstack([Q,np.zeros(env.action_space.n)])
else:
    Q = np.zeros((max_steps, env.action_space.n))

q_object=path_project+"/q_table"
fileQ=open(q_object, 'wb')
#Training the agent

#Creating lists to keep track of reward and epsilon values
total_training_rewards=0
training_rewards = []  
epsilons = []
maxi=0

for episode in range(train_episodes):
    epsilon = true_epsilon
    #Reseting the environment each time as per requirement
    state = env.reset()  
    lst=[]
    print("Epoch:"+str(episode+1)+'/'+str(train_episodes)+" epsi:"+str(epsilon))
    for step in range(max_steps-1):
        
        ### STEP 2: SECOND option for choosing the initial action - exploit     
        #If the random number is larger than epsilon: employing exploitation 
        #and selecting best action 
        if random.uniform(0, 1) > epsilon:
            action = np.argmax(Q[step,:])      
            
        ### STEP 2: FIRST option for choosing the initial action - explore       
        #Otherwise, employing exploration: choosing a random action 
        else:
            action = env.action_space.sample()
    
            
        ### STEPs 3 & 4: performing the action and getting the reward     
        #Taking the action and getting the reward and outcome state
        new_state, reward, done,trunc, info = env.step(action)
        

        ### STEP 5: update the Q-table
        #Updating the Q-table using the Bellman equation
        Q[step, action] = Q[step, action] + alpha * (reward + discount_factor * np.max(Q[step+1, :])) 
        
        total_training_rewards+=reward
        if done: break

    
        #Cutting down on exploration by reducing the epsilon 
        epsilon = epsilon*decay #find good function

     #Adding the total reward and reduced epsilon values
    training_rewards.append(total_training_rewards)
    
    if total_training_rewards > maxi:
        maxi=total_training_rewards
        print("a new reward:"+str(total_training_rewards))
        #save Q_table
        pickle.dump(Q,fileQ)
        
    total_training_rewards=0
    epsilons.append(epsilon)
fileQ.close()   
print("max score:"+str(maxi))
print("Standard deviation: "+str(np.std(training_rewards)))
print ("Mean score: " + str(sum(training_rewards)/train_episodes))


"""Lets see some good gameplay"""

state=env.reset()
lst=[]
done = False
if path.isfile(q_object):
    fileQ=open(q_object, 'rb')
    Q = pickle.load(fileQ)
    fileQ.close()
i=0
check=0
while i<max_steps-1 :
    if done: break
    lst.append(copy.deepcopy(env.render()))
    state, reward, done, _,_ =env.step(np.argmax(Q[i,:])) # take the best action
    check+=reward
    i+=1
    
print("Video_reward:"+str(check))
animation = VideoClip(make_frame, duration=len(lst)//fps) # T-second clip
# export as a video file
animation.write_videofile(video, fps=fps)

env.close()
        

2000
Epoch:1/250 epsi:0.001
a new reward:65.0
Epoch:2/250 epsi:0.001
a new reward:255.0
Epoch:3/250 epsi:0.001
Epoch:4/250 epsi:0.001
Epoch:5/250 epsi:0.001
Epoch:6/250 epsi:0.001
Epoch:7/250 epsi:0.001
Epoch:8/250 epsi:0.001
Epoch:9/250 epsi:0.001
Epoch:10/250 epsi:0.001
a new reward:410.0
Epoch:11/250 epsi:0.001
a new reward:525.0
Epoch:12/250 epsi:0.001
Epoch:13/250 epsi:0.001
Epoch:14/250 epsi:0.001
Epoch:15/250 epsi:0.001
Epoch:16/250 epsi:0.001
Epoch:17/250 epsi:0.001
Epoch:18/250 epsi:0.001
Epoch:19/250 epsi:0.001
Epoch:20/250 epsi:0.001
Epoch:21/250 epsi:0.001
Epoch:22/250 epsi:0.001
Epoch:23/250 epsi:0.001
Epoch:24/250 epsi:0.001
Epoch:25/250 epsi:0.001
Epoch:26/250 epsi:0.001
Epoch:27/250 epsi:0.001
Epoch:28/250 epsi:0.001
Epoch:29/250 epsi:0.001
Epoch:30/250 epsi:0.001
Epoch:31/250 epsi:0.001
Epoch:32/250 epsi:0.001
Epoch:33/250 epsi:0.001
Epoch:34/250 epsi:0.001
Epoch:35/250 epsi:0.001
Epoch:36/250 epsi:0.001
Epoch:37/250 epsi:0.001
Epoch:38/250 epsi:0.001
Epoch:39/250 epsi